#Pretratamiento
Este cuaderno parte de un análisis hecho. Los datos de entrada:

$$
\begin{aligned}
&\text { Training Data }\\
&\begin{array}{|l|r|}
\hline \text { Expression } & \text { Quantity } \\
\hline \text { Angry } & 3995 \\
\hline \text { Disgusted } & 436 \\
\hline \text { Fearful } & 4097 \\
\hline \text { Happy } & 7215 \\
\hline \text { Neutral } & 4965 \\
\hline \text { Sad } & 4830 \\
\hline \text { Surprised } & 3171 \\
\hline
\end{array}
\end{aligned}
$$

Dado el desbalance con las imágenes de asco (*disgusted face* )
Se hizo consulta distintas imágenes de rostros con esta expresión, almacenados en la carpeta *disgusted_faces* que serán extraidas con un modelo preentrenado en *disgusted_face*.

Comenzamos importando las librerías necesarias.

In [15]:
import cv2
import numpy as np
import random
import os
import shutil


El código es basado en funciones por tanto, se crean las funciones que habrán de emplearse.


In [36]:
# Cargar el clasificador de cascada para la detección de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def extract_faces_from_directory(input_dir, output_dir):
    """
    Función que extrae las caras de cada imagen en un directorio y las guarda en otro directorio recortadas cada cara.

    Args:
        input_dir (str): Ruta al directorio de entrada que contiene las imágenes.
        output_dir (str): Ruta al directorio de salida donde se guardarán las imágenes recortadas.

    Returns:
        None

    Raises:
        FileNotFoundError: Si el directorio de entrada no existe.
    """

    # Verificar si el directorio de entrada existe
    #if not os.path.isdir(input_dir):
        #raise FileNotFoundError("El directorio de entrada no existe.")

    # Recorrer todas las imágenes en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith(".JPG") or filename.endswith(".png"):
            # Ruta de la imagen de entrada
            input_path = os.path.join(input_dir, filename)

            # Cargar la imagen en escala de grises
            image = cv2.imread(input_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detectar rostros en la imagen
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Extraer los rostros y guardarlos en el directorio de salida
            if len(faces) > 0:
                for i, (x, y, w, h) in enumerate(faces):
                    # Ajustar los límites para asegurarse de que no se extienda más allá de la imagen original
                    x = max(0, x - 5)
                    y = max(0, y - 5)
                    w = min(w + 10, image.shape[1] - x)
                    h = min(h + 10, image.shape[0] - y)

                    # Recortar el rostro de la imagen original
                    face = image[y:y + h, x:x + w]

                    # Crear una imagen cuadrada
                    size = max(face.shape[0], face.shape[1])
                    square_face = cv2.resize(face, (size, size))

                    # Guardar la imagen en el directorio de salida
                    output_filename = f"face_{i + 1}_{filename}"
                    output_path = os.path.join(output_dir, output_filename)
                    cv2.imwrite(output_path, square_face)

                print(f"Se guardaron {len(faces)} rostros de la imagen '{filename}' en el directorio '{output_dir}'.")
            else:
                print(f"No se encontraron rostros en la imagen '{filename}'.")
                

###Reescalarlas y ponerlas en blanco y negro


def preprocess_image(image_path):
    """
    Preprocesa una imagen: convierte a blanco y negro y reescala a 48x48 píxeles.

    Args:
        image_path (str): La ruta de la imagen a procesar.

    Returns:
        numpy.ndarray: La imagen preprocesada en formato de matriz NumPy.

    Raises:
        FileNotFoundError: Si no se encuentra la imagen en la ruta especificada.

    """
    # Verificar si se encuentra la imagen
    if not os.path.exists(image_path):
        raise FileNotFoundError("No se encontró la imagen en la ruta especificada.")

    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen a blanco y negro (escala de grises)
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a 48x48 píxeles
    resized_image = cv2.resize(grayscale_image, (48, 48))

    return resized_image


##aplicar función anterior a todo un directorio

def preprocess_images_in_directory(input_dir):
    """
    Preprocesa todas las imágenes en un directorio: las convierte a blanco y negro y las redimensiona a 48x48 píxeles.

    Args:
        input_dir (str): El directorio que contiene las imágenes a procesar.

    Raises:
        FileNotFoundError: Si el directorio especificado no existe.

    """
    # Verificar si el directorio existe
    #if not os.path.isdir(input_dir):
        #raise FileNotFoundError("El directorio especificado no existe.")

    # Iterar sobre todos los archivos en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith(".JPG") or filename.endswith(".png"):
            # Obtener la ruta de entrada para la imagen
            input_path = os.path.join(input_dir, filename)

            # Preprocesar la imagen utilizando la función preprocess_image()
            processed_image = preprocess_image(input_path)

            # Obtener la ruta de salida para la imagen con extensión .png
            output_path = os.path.join(input_dir, os.path.splitext(filename)[0] + ".png")

            # Guardar la imagen procesada como .png, reemplazando el archivo original
            cv2.imwrite(output_path, processed_image)

            print(f"Imagen procesada '{filename}' guardada como '{output_path}'.")
            
            
###función para rotar y añadir ruido

import random
import cv2
import numpy as np

def rotate_and_add_noise(image):
    """
    Rota una imagen de forma aleatoria entre -45 y 45° y agrega ruido gaussiano también de forma aleatoria con una intensidad entre 0.4 y 0.65.

    Args:
        image (numpy.ndarray): La imagen a rotar y agregar ruido.

    Returns:
        numpy.ndarray: La imagen rotada y con ruido.

    """
    # Generar un ángulo de rotación aleatorio entre -45 y 45 grados
    angle = random.uniform(-45, 45)

    # Determinar el factor de escala en función de la dimensión máxima de la imagen
    max_dim = max(image.shape[0], image.shape[1])
    scale_factor = 1.0
    if max_dim > 480:
        scale_factor = 480 / max_dim

    # Redimensionar la imagen para preservar la información
    resized_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor)

    # Realizar la rotación en la imagen redimensionada
    rows, cols = resized_image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(resized_image, M, (cols, rows))

    # Generar una intensidad de ruido aleatoria entre 0.4 y 0.65
    noise_intensity = random.uniform(0.4, 0.65)

    # Generar ruido gaussiano aleatorio con la misma forma que la imagen rotada
    noise = np.random.normal(0, noise_intensity, rotated_image.shape).astype(np.uint8)

    # Agregar el ruido a la imagen rotada
    noisy_image = cv2.add(rotated_image, noise)

    return noisy_image

###función para aplicar la función anterior 3 veces a todas las imágenes en un directorio

def rotate_noise_images_in_directory(input_dir):
    """
    Preprocesa todas las imágenes en un directorio: aplica la función rotate_and_add_noise() tres veces a cada imagen, 
    convierte la imagen a blanco y negro (escala de grises) y guarda las imágenes procesadas en formato PNG.

    Args:
        input_dir (str): El directorio que contiene las imágenes a procesar.

    Raises:
        FileNotFoundError: Si el directorio especificado no existe.

    """
    # Verificar si el directorio existe
    #if not os.path.isdir(input_dir):
        #raise FileNotFoundError("El directorio especificado no existe.")

    # Iterar sobre todos los archivos en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Obtener la ruta de entrada para la imagen
            input_path = os.path.join(input_dir, filename)

            # Cargar la imagen
            image = cv2.imread(input_path)

            # Aplicar rotate_and_add_noise() tres veces a la imagen
            for i in range(3):
                processed_image = rotate_and_add_noise(image)

                # Convertir la imagen a blanco y negro (escala de grises)
                grayscale_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)

                # Obtener la ruta de salida para la imagen con extensión .png
                output_path = os.path.join(input_dir, os.path.splitext(filename)[0] + f"_processed_{i+1}.png")

                # Guardar la imagen procesada como PNG en blanco y negro
                cv2.imwrite(output_path, grayscale_image)

                print(f"Imagen procesada '{filename}' guardada como '{output_path}'.")

                


Ahora se ejecutan en secuencia las funciones creadas para que generen el tratamiento:  
1) Extraigan las caras descargadas.  
2) Se reescalen y pongan a blanco y negro.  
3) Se les aplique ruido a todas descargadas y previas, de la carpeta *train*.


In [30]:
input_dir = './pre_archive/Descargadas'
output_dir = './pre-archive/pre-tratadas'
extract_faces_from_directory(input_dir, output_dir)

Se guardaron 1 rostros de la imagen '8.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 2 rostros de la imagen '9.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 2 rostros de la imagen '14.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 4 rostros de la imagen '28.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 11 rostros de la imagen '29.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 4 rostros de la imagen '15.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 2 rostros de la imagen '17.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 9 rostros de la imagen '32..JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 1 rostros de la imagen '16.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 2 rostros de la imagen '12.JPG' en el directorio './pre-archive/pre-tratadas'.
Se guardaron 2 rostros de la imagen '13.JPG' en el directorio './pre-archive/pre

In [43]:

preprocess_images_in_directory('/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas')


###mover las imágenes de directorio
# Directorio de origen
output_dir="/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas"

# Directorio de destino
dir_destino = "/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted"

# Obtener la lista de archivos en el directorio de origen
archivos = os.listdir(output_dir)

# Filtrar solo las imágenes
imagenes = [archivo for archivo in archivos if archivo.lower().endswith((".jpg", ".png", ".jpeg"))]

# Mover las imágenes al directorio de destino
for imagen in imagenes:
    origen = os.path.join(output_dir, imagen)
    destino = os.path.join(dir_destino, imagen)
    shutil.move(origen, destino)
    print(f"Imagen {imagen} movida exitosamente.")


rotate_noise_images_in_directory(dir_destino)




Imagen procesada 'face_1_18.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_1_18.png'.
Imagen procesada 'face_1_30.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_1_30.png'.
Imagen procesada 'face_1_24.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_1_24.png'.
Imagen procesada 'face_8_2.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_8_2.png'.
Imagen procesada 'face_3_35.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_3_35.png'.
Imagen procesada 'face_4_28.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFIC

Imagen procesada 'face_2_27.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_2_27.png'.
Imagen procesada 'face_3_7.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_3_7.png'.
Imagen procesada 'face_6_30.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_6_30.png'.
Imagen procesada 'face_1_11.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_1_11.png'.
Imagen procesada 'face_7_32..png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/pre-tratadas/face_7_32..png'.
Imagen procesada 'face_10_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTI

Imagen face_4_29.png movida exitosamente.
Imagen face_3_34.png movida exitosamente.
Imagen face_4_15.png movida exitosamente.
Imagen face_1_25.png movida exitosamente.
Imagen face_1_31.png movida exitosamente.
Imagen face_8_29.png movida exitosamente.
Imagen face_3_32..png movida exitosamente.
Imagen face_1_27.png movida exitosamente.
Imagen face_1_33.png movida exitosamente.
Imagen face_8_1.png movida exitosamente.
Imagen face_1_26.png movida exitosamente.
Imagen face_1_22.png movida exitosamente.
Imagen face_3_33.png movida exitosamente.
Imagen face_10_29.png movida exitosamente.
Imagen face_1_23.png movida exitosamente.
Imagen face_1_35.png movida exitosamente.
Imagen face_1_21.png movida exitosamente.
Imagen face_15_33.png movida exitosamente.
Imagen face_3_30.png movida exitosamente.
Imagen face_4_10.png movida exitosamente.
Imagen face_3_31.png movida exitosamente.
Imagen face_2_32..png movida exitosamente.
Imagen face_6_29.png movida exitosamente.
Imagen face_1_20.png movida exi

Imagen procesada 'im78.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im78_processed_3.png'.
Imagen procesada 'im163.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im163_processed_1.png'.
Imagen procesada 'im163.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im163_processed_2.png'.
Imagen procesada 'im163.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im163_processed_3.png'.
Imagen procesada 'im177.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im177_processed_1.png'.
Imagen procesada 'im177.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFI

Imagen procesada 'face_1_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_1_33_processed_2.png'.
Imagen procesada 'face_1_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_1_33_processed_3.png'.
Imagen procesada 'im202.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im202_processed_1.png'.
Imagen procesada 'im202.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im202_processed_2.png'.
Imagen procesada 'im202.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im202_processed_3.png'.
Imagen procesada 'im216.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/I

...

Imagen procesada 'im395.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im395_processed_2.png'.
Imagen procesada 'im395.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im395_processed_3.png'.
Imagen procesada 'im73.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im73_processed_1.png'.
Imagen procesada 'im73.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im73_processed_2.png'.
Imagen procesada 'im73.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im73_processed_3.png'.
Imagen procesada 'im98.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/

Imagen procesada 'im232.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im232_processed_3.png'.
Imagen procesada 'face_6_2.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_6_2_processed_1.png'.
Imagen procesada 'face_6_2.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_6_2_processed_2.png'.
Imagen procesada 'face_6_2.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_6_2_processed_3.png'.
Imagen procesada 'im88.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im88_processed_1.png'.
Imagen procesada 'im88.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/IN

Imagen procesada 'im179.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im179_processed_2.png'.
Imagen procesada 'im179.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im179_processed_3.png'.
Imagen procesada 'face_10_34.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_10_34_processed_1.png'.
Imagen procesada 'face_10_34.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_10_34_processed_2.png'.
Imagen procesada 'face_10_34.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_10_34_processed_3.png'.
Imagen procesada 'im62.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2

Imagen procesada 'face_12_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_12_33_processed_1.png'.
Imagen procesada 'face_12_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_12_33_processed_2.png'.
Imagen procesada 'face_12_33.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_12_33_processed_3.png'.
Imagen procesada 'face_6_1.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_6_1_processed_1.png'.
Imagen procesada 'face_6_1.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/face_6_1_processed_2.png'.
Imagen procesada 'face_6_1.png' guardada como '/Users/macbook/Documents/E

Imagen procesada 'im387.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im387_processed_3.png'.
Imagen procesada 'im49.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im49_processed_1.png'.
Imagen procesada 'im49.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im49_processed_2.png'.
Imagen procesada 'im49.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im49_processed_3.png'.
Imagen procesada 'im350.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL/Proyecto/pre_archive/disgusted/im350_processed_1.png'.
Imagen procesada 'im350.png' guardada como '/Users/macbook/Documents/ESPECIALIZACIÓN/2do SEMESTRE/INTELIGENCIA ARTIFICIAL

Con lo anterior se tienen para entrenamiento:  
$$
\begin{aligned}
&\text { Training Data }\\
&\begin{array}{|l|r|}
\hline \text { Expression } & \text { Quantity } \\
\hline \text { Angry } & 3995 \\
\hline \text { Disgusted } & 2348 \\
\hline \text { Fearful } & 4097 \\
\hline \text { Happy } & 7215 \\
\hline \text { Neutral } & 4965 \\
\hline \text { Sad } & 4830 \\
\hline \text { Surprised } & 3171 \\
\hline
\end{array}
\end{aligned}
$$
